In [2]:
import glob
import scipy
import os,sys
import zipfile
import numpy as np
import nibabel as nib
import tensorflow as tf
from scipy import ndimage
from tensorflow import keras
from natsort import natsorted
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from aifnet_utils.preprocess import read_nifti_file, normalize, normalize_aif, process_scan, normalize_zero_one
from aifnet_utils.data_loaders import read_isles_volumepaths_from_file_otf, read_isles_annotations_from_file, ISLES18DataGen_aifvof_otf
from aifnet_utils.data_loaders import delay_sequence_padding, anticipate_sequence_padding, late_bolus, early_bolus
from aifnet_utils.results import plot_predictions
from aifnet_utils.losses import MaxCorrelation
from scipy import signal
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from scipy.linalg import convolution_matrix, toeplitz, circulant
from sklearn.linear_model import Ridge
from matplotlib import pyplot, image, transforms
from scipy import ndimage
from aifnet_utils.models_aifnet import get_model_twoPvols
from numpy import inf
from aifnet_utils.results import plot_predictions

import random
keras.backend.set_image_data_format('channels_last')

%matplotlib inline
!pwd

/Users/sebastianotalora/work/postdoc/ctp/aifnet_replication


In [3]:
#Reading an example PCT volume
LOCATION = 'LOCAL'
if LOCATION == 'LOCAL':
    ROOT_EXP = '/Users/sebastianotalora/work/postdoc/ctp/aifnet_replication/'
    root_dir  = '/Users/sebastianotalora/work/postdoc/data/ISLES/'

if LOCATION == 'INSEL':
    ROOT_EXP = '/home/sebastian/experiments/aifnet_replication/'
    root_dir  = '/media/sebastian/data/ASAP/ISLES2018_Training'

aif_annotations_path = ROOT_EXP + 'radiologist_annotations.csv'



In [4]:
min_num_volumes_ctp = 43

In [5]:
def compute_predictions_aif(model_prefix):
    model_prefix_meassures = []
    for current_fold in range(1,6):
        prediction_meassures = []
        #Reading AIFs and VOFs for each of the partitions
        test_partition_path =  ROOT_EXP+'/partitions/fold_'+str(current_fold) +'/test_v2.txt'

        aif_annotations_test, vof_annotations_test = read_isles_annotations_from_file(aif_annotations_path,  test_partition_path,
                                                root_dir, min_num_volumes_ctp, return_aif_only = False)
        ctp_volumes_test = read_isles_volumepaths_from_file_otf(root_dir, test_partition_path, aif_annotations_path)
        print('======= PREDICTING USING '+ model_prefix +' FOR TEST PARTITION FOR THE FOLD ' + str(current_fold) + ' =======')
        print(len(ctp_volumes_test), len(aif_annotations_test))
        modelweights_path= ROOT_EXP + 'results/trained_models/aifnet_2Pvols_SGD_MaxCorr_augment_lr/'+ model_prefix+'_fold_'+str(current_fold)+'.hdf5'
        model = get_model_twoPvols(width=256, height=256, num_channels=min_num_volumes_ctp)
        model.load_weights(modelweights_path)
        results_meassures, prediction_ids = [], []
        for case_number in range(len(ctp_volumes_test)):
            case_id = ctp_volumes_test[case_number]['image'].split('.')[-2]
            prediction_ids.append(case_id)
            cur_nib = nib.load(ctp_volumes_test[case_number]['image'])
            ctp_vals = cur_nib.get_fdata()
            x = normalize(ctp_vals[:,:,:,0:min_num_volumes_ctp])
            if type_predictions == 'AIF':
                y = aif_annotations_test[case_id]
            if type_predictions == 'VOF':
                y = vof_annotations_test[case_id]
            prefix_fig = ROOT_EXP + '/results/predictions_aif/'+modelweights_path.split('/')[-1]+'_case_'+str(case_id)
            results_meassures.append(plot_predictions(model,x,y, prefix_fig, normalize_preds=True, type_pred=type_predictions, savefig=True))

        preds_fold = tfp.stats.correlation(np.array(results_meassures)[:,1,:],np.array(results_meassures)[:,0,:], sample_axis=0, event_axis=None)
        preds_fold = preds_fold.numpy()
        prediction_meassures.append([preds_fold.mean(),preds_fold.std(),preds_fold.var()])
        model_prefix_meassures.append(prediction_meassures)
        np.savetxt('results/'+model_prefix+'pearson_fold_'+str(current_fold)+'.csv', prediction_meassures, delimiter=',',fmt='%1.5f')
        np.savetxt('results/'+model_prefix+'allpreds_fold_'+str(current_fold)+'.csv', np.array(results_meassures)[:,1,:], delimiter=',',fmt='%1.5f')

        test_ids_file=open('results/'+model_prefix+'pred_ids_fold_'+str(current_fold)+'.csv','w')
        for element in prediction_ids:        
            test_ids_file.write(element+'\n')
        test_ids_file.close()
    model_prefix_meassures = np.array(model_prefix_meassures)
    np.savetxt('results/'+model_prefix+'pearson_ALLFOLDS.csv', model_prefix_meassures.mean(axis=0), delimiter=',',fmt='%1.5f')
    return model_prefix_meassures.mean(axis=0)

In [6]:
last_trained_model_paths = glob.glob('/Users/sebastianotalora/work/postdoc/ctp/aifnet_replication/results/trained_models/aifnet_2Pvols_SGD_MaxCorr_augment_lr/*')
last_prefixes = []
for item in last_trained_model_paths:
    prefix_path = item.split('/')[-1].split('_fold')[0]
    if prefix_path not in set(last_prefixes):
        last_prefixes.append(prefix_path)
print(len(last_prefixes))
print(len(last_trained_model_paths))

15
75


In [7]:
type_predictions = 'AIF'
for model_prefix in last_prefixes:
    print(compute_predictions_aif(model_prefix))

======= PREDICTING USING aifnet_2Pvols_SGD_MaxCorr_augment_lr0.9623209115272406 FOR TEST PARTITION FOR THE FOLD 1 =======
31 31


OSError: Unable to open file (unable to open file: name = '/Users/sebastianotalora/work/postdoc/ctp/aifnet_replication/results/trained_models/models_14_09_21/aifnet_2Pvols_SGD_MaxCorr_augment_lr0.9623209115272406_fold_1.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [26]:
last_prefixes

['aifnet_2Pvols_SGD_mse_augment_lr0.8923848003155606',
 'aifnet_2Pvols_SGD_mse_augment_lr5.197918264960655e-06',
 'aifnet_2Pvols_SGD_mse_augment_lr0.26664456007954873',
 'aifnet_2Pvols_SGD_mse_augment_lr0.24466717332886467',
 'aifnet_2Pvols_SGD_mse_augment_lr0.8286085065732829',
 'aifnet_2Pvols_SGD_mse_augment_lr0.019013083285230357',
 'aifnet_2Pvols_SGD_mse_augment_lr0.0012656176411610413',
 'aifnet_2Pvols_SGD_mse_augment_lr0.6930557976225901',
 'aifnet_2Pvols_SGD_mse_augment_lr5.156127099319659e-06',
 'aifnet_2Pvols_SGD_mse_augment_lr0.31965383057547536',
 'aifnet_2Pvols_SGD_mse_augment_lr0.32219260475608635',
 'aifnet_2Pvols_SGD_mse_augment_lr0.18557241904693844',
 'aifnet_2Pvols_SGD_mse_augment_lr0.41756190770883805',
 'aifnet_2Pvols_SGD_mse_augment_lr0.04584660141746011',
 'aifnet_2Pvols_SGD_mse_augment_lr0.14205136895431247']